<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/chainlit_faiss_llm_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MEDICAL CHATBOT
CHAINLIT+PDF+FAISS_DB+LLM

## Initial environment settings

In [ ]:
# Initial environment settings
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

## Confirm your CUDA version and whether you can use a GPU.


In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

Wed Nov 26 15:31:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             32W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

True

## Installation kit

In [ ]:
!pip install cohere gdown>=4.7.3 kaleido langchain langchain-community langchain-text-splitters langchain-core langchain-openai chainlit pyngrok pypdf python-dotenv sentence-transformers tiktoken -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires starlette<1.0.0,>=0.46.2, but you have starlette 0.41.3 which is incompatible.


In [ ]:
!pip install accelerate bitsandbytes faiss-cpu faiss-gpu hf_transfer huggingface_hub optimum transformers  -q

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
!pip install --upgrade langgraph==0.2.74 langchain_core==0.3.37 langchain_openai==0.3.6 langchain==0.3.19 langchain_community==0.3.18 chainlit==2.5.5 --force-reinstall -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 90.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gptqmodel 5.4.2 requires numpy==2.2.6, but you have numpy 2.3.5 which is incompatible.
cohere 5.20.0 requires httpx-sse==0.4.0, but you have httpx-sse 0.4.3 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.37 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.6 which is incompatible.
opentelemetry-instrumentation-agno 0.49.3 requires opentelemetry-instrumentation>=0.59b0, but you have opentelemetry-instrumentation 0.58b0 which is incompatible.
opentelemetry-instrumentation-agno 0.49.3 requires opentelemetry-semantic-conventions>=0.59b0, but you have opentelemetry-semantic-conventions 0

In [ ]:
!pip install auto-gptq -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install gptqmodel -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.1 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.5 which is incompatible.


### File download

In [ ]:
!mkdir -p data/book/
!gdown 1pUDgs3YMnlr8See8Rld3L1ZRiTeeOlMM -O data/book/

Downloading...
From: https://drive.google.com/uc?id=1pUDgs3YMnlr8See8Rld3L1ZRiTeeOlMM
To: /content/data/book/Medical_Chatbot.pdf
100% 16.1M/16.1M [00:00<00:00, 40.6MB/s]


## Step1: create ingest.py

In [ ]:
%%bash
cat << \EOF >  ingest.py

# RUN: python3 ingest.py

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

DATA_PATH = 'data/book/'
DB_FAISS_PATH = 'vectorstore/db_faiss'

# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                   chunk_overlap=100)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(
      model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
      model_kwargs={'device': 'cuda'} # else cpu
    )

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

if __name__ == "__main__":
    create_vector_db()

EOF

### Vector storage

In [ ]:
# Vector storage to FAISS
!python3 ingest.py

/content/ingest.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2025-11-26 15:35:55.273027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764171355.309043   14584 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764171355.319143   14584 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regi

## Step2: create model.py

In [ ]:
%%bash
cat << \EOF >  model.py
# RUN: chainlit run model.py

# 01: CONFIGURE MODEL_ID and DB PATHs
#MODEL_ID = "TheBloke/Llama-2-7b-Chat-GPTQ"
MODEL_ID = "google/gemma-3-1b-it"
DB_FAISS_PATH = 'vectorstore/db_faiss'

# 02: Load LIBRARY
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.retrieval_qa.base import RetrievalQA
import chainlit as cl
import torch
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
#warnings.filterwarnings('ignore')

# 03: custom_prompt_template
custom_prompt_template = """
You are a medical bot.
Use the following pieces of information (Context) to answer the user's Question in Spanish.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else. Always answer in Spanish.
Helpful answer:
"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

# 04: Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 5}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

def load_llm():
    # 04: Full LLM Model or GPTQ
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, device_map="auto")
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        return_full_text=False,
    )
    llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.7})

    return llm

# 05: QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", model_kwargs={'device': 'cuda'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

# 06: output function
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

# 07: chainlit code
@cl.on_chat_start
async def start():
    chain = qa_bot()
    msg = cl.Message(content="Starting the bot...")
    await msg.send()
    msg.content = "Hi, Welcome to Medical Bot. What is your query?"
    await msg.update()

    cl.user_session.set("chain", chain)

@cl.on_message
async def main(message: cl.Message):
    chain = cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler(
        stream_final_answer=True, answer_prefix_tokens=["FINAL", "ANSWER"]
    )
    cb.answer_reached = True
    res = await chain.acall(message.content, callbacks=[cb])
    answer = res["result"]
    sources = res["source_documents"]
    source_elements = []

    found_sources = []

    for i, doc in enumerate(sources):
        page_content=(res['source_documents'][i].page_content)
        page=(res['source_documents'][i].metadata["page"])
        source=res['source_documents'][i].metadata["source"]
        file = os.path.basename(source)
        #print("SOURCE: "+file+", PAGE: "+str(page) )
        source_name=f"source_{i}"

        found_sources.append(source_name)
        text="[Document: "+file+", Page: "+str(page)+"]\n\nContent: "+page_content
        source_elements.append(cl.Text(content=text, name=source_name))

    if sources:
        #answer += f"\n\n\nSource of information, Sources:" + str(sources)
        answer += f"\nSources: {', '.join(found_sources)}"
    else:
        answer += "\n\n\nNo sources found"

    #await cl.Message(content=answer).send()
    await cl.Message(content=answer, elements=source_elements).send()

EOF

## Execution Model

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CHAINLIT
!chainlit run model.py -w &> /content/logs.txt &

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://8000-gpu-t4-s-1zuzb2smm4xia-a.us-west4-2.prod.colab.dev


## ngrok (optional)

In [ ]:
!ngrok config add-authtoken xxx

from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

### DELETE JOB, Execute again before the end

In [ ]:
ngrok.kill()

In [ ]:
!ps -ef |grep chainlit | awk '{print $2}' | xargs kill -9
!ps -ef |grep ngrok | awk '{print $2}' | xargs kill -9
